In [10]:
import os
import argparse

import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm
from python_speech_features import mfcc
# from HMMTrainer import HMMTrainer
import warnings

import tensorflow as tf
# Recurrent Neural Network
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense , LSTM
# from tensorflow.keras.layers.recurrent import LSTM

In [11]:
import IPython
from scipy.io import wavfile
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.signal import butter, lfilter
%matplotlib inline


In [12]:

def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)


In [13]:
def  SVDnoise(src_data):
    A=_stft(src_data,512,64,512)
    U,S,Vt=np.linalg.svd(A)
    Snew=np.zeros(np.shape(A))
    for i in range(len(S)):
        if i<10:
            Snew[i,i]=S[i]
    C=np.matmul(U,Snew)
    Anew=np.matmul(C,Vt)
    x=_istft(Anew,64,512)
    return x


In [14]:
def S1S2(x):
    x=x/max(np.abs(x))
    s=-1*(x*x)*np.log(x*x)
    a=np.ones(201)
    ss=np.convolve(s,a)
    ss=ss/np.max(ss)
    x2=x*(ss>0.2)[:len(x)]
    x2=x2+0.0000000000000000001
    senv=-1*(x2*x2)*np.log(x2*x2)
    a=np.ones(201)
    senv=np.convolve(senv,a)
    senv=senv/np.max(ss)
    sp=scipy.signal.hilbert(senv)
    ph=np.abs(np.angle(senv+(1j)*sp))
    der=np.zeros(len(ph))
    start=[]
    enda=[]
    flag=0
    for i in range(len(senv)-1):
        der[i]=ph[i+1]-ph[i]
        if der[i]>0 and  ph[i]>0.1 and flag == 0  :
            start.append(i)
            flag=1
        if der[i]<-2.0 :
            flag=0
    flag=0        
    for i in range(len(senv)-1):
        if der[i]>0 and  ph[i]>3.0 and flag == 0  :
            enda.append(i)
            flag=1
        if der[i]<-2.0 :
            flag=0
#     print((start))
#     if enda[0] < start[0]:
#         enda = enda[1:]
#     if enda[-1] < start[-1]:
#         start = start[0:len(enda)]
    return (start,enda)


In [15]:
def S1_S2_classifier(signal, start_index, end_index, sampling_rate):              

    I = []#np.array() # kth element will tell the length of the kth interval
    D = []#np.array() # kth element will tell the duration of lub/dub in the kth interval
    A = []#np.array() # sum of all values of the signal(abs) in the kth duration
    for x in range (0, len(start_index)-1):
        i_kth = start_index[x+1] - start_index[x]
        I.append(i_kth)
    print(I)
    print(start_index)
    print(end_index)
    #check for first element in end_index
    
#     if end_index[0] < start_index[0]:
#         np.delete(end_index, 0)

    for x in range (0, len(start_index)-1):
        d_kth = end_index[x] - start_index[x]
        # length of start_index != length of end_index in 2nd iteration case
        D.append(d_kth)
    print(D)
    print('jhkjhj')
    print(len(start_index), len(end_index), len(D), len(A), len(I))
    #print('jhkjhj')
    #print(len(D))
    #print('jhkjhj')
    #print(len(I))
    #print('jhkjhj')
    #print(len(A))
    print()
    sig = np.absolute(signal)
    #print(sig)
    for x in range (0, len(start_index)):
        sum = 0
        for y in range (start_index[x],end_index[x]):
            sum += sig[y]
        A.append(sum)
    ### All the arrays I, D and A have been constructed ###
    label = []#np.array()
    if(len(I)==0 or len(D)==0 or len(A)==0):
        return([1])
    for k in range (0, len(start_index)):
        if I[k] < I[k+1]:
            if (I[k+1]-I[k] > (0.05*sampling_rate)):
                label.append(1)
            elif (D[k] > D[k+1]) and ((D[k]-D[k+1]) > 0.005*sampling_rate):
                label.append(1)
            elif (A[k] > 1.5*A[k+1]):
                label.append(1)
        else:
            label.append(2)
        if I[k] > I[k+1]:
            if (I[k]-I[k+1] > (0.05*sampling_rate)):
                label.append(2)
            elif (D[k] < D[k+1]) and ((D[k+1]-D[k]) > 0.005*sampling_rate):
                label.append(2)
            elif (A[k] < 1.5*A[k+1]):
                label.append(2)
        else:
            label.append(1)
        # if I[k] == I[k+1] ??
    #print(start_index)            
        
   
    return label


In [ ]:
time_steps = 150
visible=Input(shape=(time_steps,13))
hidden1 = LSTM(10,return_sequences=True)(visible)
hidden2 = LSTM(10)(hidden1)
hidden3 = Dense(10, activation='relu')(hidden2)
# hidden3 = Dense(10, activation='relu')(hidden2)
output = Dense(1, activation='sigmoid')(hidden3)
model = Model(inputs=visible, outputs=output)

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

class heart_sound_classifier:

    digit_directory = '/home/naman/Desktop/HeartData/train-2'
    test_directory = '/home/naman/Desktop/HeartData/test'
    nfft = 1203 # Number of FFTs
#     print(digit_directory)
    def run(self):

        processed_files = list()
        mfcc_features = self.process_directory()
        train_size = 0
        for feature in mfcc_features:
            train_size += 1
            processed_files.append({'label': feature["label"], 'feature': feature["mfcc"]})
        y_train = np.zeros((train_size))
        x = np.zeros((train_size, time_steps ,13))
    # Train rnn for each MFCC and add to training set
        i = 0
        for processed_file in processed_files:
#             print(processed_file['label'])
#             print(processed_file['feature'].shape)
            x[i,:,:] = processed_file['feature']
            s = processed_file['label']
            if(s[0]=='a'):
                y_train[i]=1
            else:
                y_train[i]=0
            i += 1
        normalize(x)
#         print(y_train)
        model.compile(loss='logcosh', optimizer='adam', metrics=['accuracy'])
        model.fit(x, y_train, epochs=30, batch_size=10)

    def process_directory(self):
        mfcc_features = list()
        for filename in [x for x in os.listdir(self.digit_directory) if x.endswith('.wav')]:
            # Read the input file
            filepath = os.path.join(self.digit_directory, filename)
            print(filepath)
            sampling_freq, audio = wavfile.read(filepath)
#             label = self.get_label(filename)
            label="n.wav"
            # Extract MFCC features and append to list
            st="/home/naman/Desktop/HeartData/train-2"+"/"+filename.split(".")[0]+".hea"
            with open(st,'r') as f:
                for line in f:
                    for word in line.split():
                        if(word=="Abnormal"):
                            label="a.wav"
#                         print(word)
#             print(label)
            audio1 = audio
##################################################### Segment extraction  ##########################################################
            if(len(np.shape(audio))!=1):
                audio1=SVDnoise(audio[0,:])
            else:
                audio = np.asarray(audio, dtype = float)
                audio1=SVDnoise(audio)
            start,enda = S1S2(audio1)
#             print(start)
            if len(start) <= 5:
#                 print('hello')
                continue
#             #print(len(start))
#             labels = S1_S2_classifier(audio1,start,enda,sampling_freq)
# #             index = 0
#             for index in range(0,len(labels)):
#                 if labels[index]==1:
#                     audio1 = audio1[start[index]:]
#                     break
###################################################################################################################################
            audio1 =audio1[start[3]:]
            temp = mfcc(audio1, sampling_freq, nfft=self.nfft)
            temp = temp[0:time_steps,:]
            mfcc_features.append({"label": label, "mfcc": temp})
        return mfcc_features
    
#     def get_label(self, filename):
#         filename=filename.split("-")[1]
# #         print(filename)
#         return filename
    

if __name__ == "__main__":
    dr = heart_sound_classifier()
    dr.run()

In [ ]:
def test_direc():
    mfcc_features = list()
    for filename_test in [x1 for x1 in os.listdir(test_directory) if x1.endswith('.wav')]:
        # Read the input file
        filepath = os.path.join(test_directory, filename_test)
        sampling_freq, audio = wavfile.read(filepath)
#       label = self.get_label(filename1)
        label="n.wav"
        st="/home/naman/Desktop/HeartData/test-2"+"/"+filename_test.split(".")[0]+".hea"
        with open(st,'r') as f:
            for line in f:
                for word in line.split():
                    if(word=="Abnormal"):
                        label="a.wav"
        # Extract MFCC features and append to list
        temp1 = mfcc(audio, sampling_freq, nfft=1203)
        temp1 = temp1[0:time_steps,:]
        mfcc_features.append({"label": label, "mfcc": temp1})
    return mfcc_features

test_directory = '/home/naman/Desktop/HeartData/test-2'
test_files = list()
test_features = test_direc()
test_size = 0
for feature in test_features:
    test_size += 1
    test_files.append({'label': feature["label"], 'feature': feature["mfcc"]})
y_test = np.zeros((test_size))
x_test = np.zeros((test_size, time_steps ,13))
i = 0

for test_file in test_files:
    x_test[i,:,:] = test_file['feature']
    s = test_file['label']
#             print(s)
    if(s[0]=='a'):
        y_test[i]=1
    else:
        y_test[i]=0
    i += 1
normalize(x_test)
predict=model.predict(x_test)
print(predict)
model.evaluate(x_test,y_test)
print(y_test)
        

    
# def get_label(self, filename):
#     filename=filename.split("-")[1]
# #         print(filename)
#     return filename

        
        